In [56]:
#Here info:
#App ID: gB5Th31Aa7kCMDRB2yYH
#App Code: _V3OMdtHzj0vDll0eT6JsA

from sklearn.ensemble import RandomForestClassifier
from sklearn import (metrics, model_selection, linear_model, preprocessing, ensemble, neighbors)
from sklearn.neighbors import KNeighborsClassifier
import matplotlib.pyplot as plt
from matplotlib import rcParams
import seaborn as sns
import numpy as np
import pandas as pd
import pprint as pp
import re
from sklearn.model_selection import train_test_split
#import xgboost as xgb



In [120]:
# import the "crash" data
data = pd.read_csv("my_map_grid.csv",header=7)

# preprocessing
# lowercase: http://stackoverflow.com/a/38931854
data.columns = data.columns.str.lower()
# remove spaces
data.columns = data.columns.str.replace(' ', '_')
# special cases
data.columns = data.columns.str.replace('crash_i_d', 'crash_id')
# remove whateva data

# replace ['No Data','Not Applicable'] with NaN
data.replace(to_replace='No Data', value=np.nan, inplace=True)
data.replace(to_replace='Not Applicable', value=np.nan, inplace=True)

dummies_needed_list = [
 'day_of_week',
 'intersection_related',
 'light_condition',
 'manner_of_collision',
 'road_base_type',
 'surface_condition',
 'weather_condition'
        ]
# encode data for dummies_needed_list
for feat in dummies_needed_list:
    data = pd.concat([data,pd.get_dummies(data[feat],prefix=feat)],axis=1)
    data = data.drop([feat],axis=1)

#remove data that will not be usefull
drop_columns = ['crash_id',
                'average_daily_traffic_amount',
                'average_daily_traffic_year',
                'street_name',
                'intersecting_street_name',
                'medical_advisory_flag',
                'object_struck']
crash_id = data.crash_id
data = data.drop(drop_columns,axis=1)

#fill missing values
data = data.fillna(data.mean())

#split data into train/test,x/y
np.random.seed(0)
train, test = train_test_split(data)
x_train = train.drop(['crash_severity'],axis=1)
y_train = train.crash_severity
x_test = test.drop(['crash_severity'],axis=1)
y_test = test.crash_severity

In [95]:
#test models from data will need a model that can classify more than 2 response classes
knn = KNeighborsClassifier()
knn.fit(x_train,y_train)

ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

In [46]:
import urllib.request
urllib.request.urlopen('http://reverse.geocoder.cit.api.here.com/6.2/reversegeocode.json?prox=29.42458%2C+-98.49461&app_id=gB5Th31Aa7kCMDRB2yYH&app_code=_V3OMdtHzj0vDll0eT6JsA&mode=retrieveAddresses&gen=8&language=').read()

b'{"Response":{"MetaInfo":{"Timestamp":"2017-04-28T19:09:10.237+0000"},"View":[{"_type":"SearchResultsViewType","ViewId":0,"Result":[{"Relevance":1.0,"Distance":0.0,"MatchLevel":"street","MatchQuality":{"Country":1.0,"State":1.0,"County":1.0,"City":1.0,"District":1.0,"Street":[1.0],"PostalCode":1.0},"Location":{"LocationId":"NT_i0k6XnW3CNq2ac8vLrKDuC_l_843956229_L","LocationType":"address","DisplayPosition":{"Latitude":29.42458,"Longitude":-98.49461},"NavigationPosition":[{"Latitude":29.42458,"Longitude":-98.49461}],"MapView":{"TopLeft":{"Latitude":29.42469,"Longitude":-98.49462},"BottomRight":{"Latitude":29.42447,"Longitude":-98.4946}},"Address":{"Label":"S Flores St, San Antonio, TX 78205, United States","Country":"USA","State":"TX","County":"Bexar","City":"San Antonio","District":"Downtown","Street":"S Flores St","PostalCode":"78205","AdditionalData":[{"value":"United States","key":"CountryName"},{"value":"Texas","key":"StateName"},{"value":"Bexar","key":"CountyName"},{"value":"N","

In [43]:
import geocoder

g = geocoder.here([29.42458,-98.49461],app_id='gB5Th31Aa7kCMDRB2yYH',app_code='_V3OMdtHzj0vDll0eT6JsA',method='reverse',mode='retrieveAll',responseattributes='SpeedLimitType')
g.url

'http://reverse.geocoder.cit.api.here.com/6.2/reversegeocode.json?prox=29.42458%2C+-98.49461&app_id=gB5Th31Aa7kCMDRB2yYH&app_code=_V3OMdtHzj0vDll0eT6JsA&mode=retrieveAddresses&gen=8&language='

In [ ]:
# convert 'Wet' 'Dry' to '1' '0'
data['surface_condition'] = data['surface_condition'].factorize()[0]
# DOC: rename col http://stackoverflow.com/a/11346337
data.rename(columns={'surface_condition':'surface_wet'})
# print number of unique
for colname in data.columns:
    print("% 4d : %s" % (len(data[colname].unique()), colname))
# remove data which is has no importance
# better to drop cols with all NaN and convert "unimportant" data to NaN
#  - can't universally decide to drop col just based on uniqueness
# e.g. all of object_struck is 'Not Applicable' and useless, but if surface_condition had only one value "dry" this would be important
# ? for colname in data.columns:
# colname = 'object_struck'
# if(len(data[colname].unique()) == 1):
#   print("-I-: dropping %s for having all homogenous values %s", (colname, data[colname].unique()[0]))
#   data.drop(colname,axis=1,inplace=True)


    

print(data.head())
print(data.info())
if(1):
  data.describe()
  data.hist()
  data.corr().plot() # TODO: seaborn
  plt.show()
else:
  print("-I-: Skipping...")

pairplot_var_list = [
# 'crash_id',
 'average_daily_traffic_amount',
 'average_daily_traffic_year',
 'crash_death_count',
# 'crash_incapacitating_injury_count',
# 'crash_non-incapacitating_injury_count',
# 'crash_not_injured_count',
# 'crash_possible_injury_count',
 'crash_severity',
 'crash_time',
 'crash_year',
 'day_of_week',
# 'intersecting_street_name',
 'intersection_related',
# 'latitude',
 'light_condition',
# 'longitude',
 'manner_of_collision',
 'medical_advisory_flag',
 'number_of_entering_roads',
 'number_of_lanes',
# 'object_struck',
 'road_base_type',
 'speed_limit',
# 'street_name',
 'surface_condition'
 ]

# tmp disable
if(0):
    sns.pairplot(data, vars=pairplot_var_list)
    plt.show()

# alternative visualisation
datapt = data.pivot_table(values=['crash_death_count','crash_incapacitating_injury_count','crash_non-incapacitating_injury_count'], index=['speed_limit','crash_time'])
print(datapt)

pp.pprint(list(pd.get_dummies(data[dummies_needed_list]).columns))
pp.pprint(list(pd.get_dummies(data[dummies_needed_list]).columns.str.replace('[,\s]+','_').str.lower()))
'''
 'Dark, Lighted', 'dark_lighted_yes'
 'Dark, Not Lighted', 'dark_lighted_no'
 'Dark, Unknown Lighting', 'dark_lighted_unknown'
 'Dawn',
 'Daylight',
 'Dusk',
 'Unknown',
'''